# **Lab 1 การวิเคราะห์พื้นที่ให้บริการของสถานีขนส่งสาธารณะด้วย GeoPandas**
การวิเคราะห์ข้อมูลเชิงพื้นที่เป็นองค์ประกอบสำคัญของงานระบบสารสนเทศภูมิศาสตร์ (GIS) โดยเฉพาะในการวางแผนและประเมินโครงสร้างพื้นฐานด้านการคมนาคมขนส่ง ห้องปฏิบัติการนี้มีวัตถุประสงค์เพื่อศึกษาการประยุกต์ใช้เครื่องมือ GeoPandas สำหรับการวิเคราะห์พื้นที่ให้บริการของสถานีขนส่งสาธารณะ ผ่านกระบวนการสร้าง Buffer และ Spatial Join เพื่อประเมินการครอบคลุมพื้นที่เมือง

การปฏิบัติการครั้งนี้ช่วยพัฒนาทักษะการจัดการข้อมูลเชิงพื้นที่ การแปลงระบบพิกัด การคำนวณพื้นที่ให้บริการ และการสื่อสารผลลัพธ์ผ่านแผนที่แบบ Interactive ซึ่งเป็นพื้นฐานสำคัญของการวิเคราะห์เชิงพื้นที่ในงาน GIS สมัยใหม่

## **ข้อมูลที่ใช้ในการศึกษา**
ใช้ข้อมูลตำแหน่งสถานีขนส่งสาธารณะในเขตกรุงเทพมหานคร ซึ่งดึงมาจาก OpenStreetMap ผ่านไลบรารี osmnx โดยข้อมูลอยู่ในรูปแบบ Point Geometry และถูกนำมาวิเคราะห์ร่วมกับขอบเขตพื้นที่เมือง

### **ขั้นตอนการดำเนินงาน**

ขั้นตอนที่ 1 ติดตั้งไลบรารี

In [87]:
# ติดตั้งเครื่องมือสำหรับจัดการข้อมูล GIS และสร้างแผนที่
!pip install geopandas folium osmnx

ขั้นตอนที่ 2 นำเข้าไลบรารี

In [88]:
# โหลดไลบรารีที่ใช้วิเคราะห์และแสดงผล
import geopandas as gpd
import folium
import osmnx as ox

ขั้นตอนที่ 3 โหลดข้อมูลสถานีขนส่ง (กรุงเทพมหานคร)

In [89]:
# ดึงข้อมูลสถานีขนส่งจาก OpenStreetMap และกำหนดระบบพิกัด
place = "Bangkok, Thailand"
tags = {"amenity": "bus_station"}

stations = ox.features_from_place(place, tags)
stations = stations.reset_index()
stations = gpd.GeoDataFrame(stations, geometry="geometry", crs="EPSG:4326")

ขั้นตอนที่ 4 แปลงระบบพิกัดและสร้าง Buffer

In [90]:
# แปลง CRS เป็นหน่วยเมตรเพื่อสร้าง Buffer ระยะ 5 กิโลเมตร
stations_proj = stations.to_crs(epsg=32647)
stations_proj["buffer"] = stations_proj.geometry.buffer(5000)
buffer_gdf = stations_proj.set_geometry("buffer")

ขั้นตอนที่ 5 วิเคราะห์พื้นที่ครอบคลุม (Spatial Join)

In [91]:
# ตรวจสอบความสัมพันธ์เชิงพื้นที่ของข้อมูล
boundary = stations_proj.buffer(0).unary_union.convex_hull
boundary_gdf = gpd.GeoDataFrame(geometry=[boundary], crs=stations_proj.crs)

join = gpd.sjoin(stations_proj, boundary_gdf, how="left")

ขั้นตอนที่ 6 สร้าง Interactive Map

In [92]:
# แปลง CRS กลับ lat/long
stations_wgs = stations_proj.to_crs(4326)
buffer_wgs = buffer_gdf.to_crs(4326)

# ===== เตรียม buffer สำหรับ plot =====
buffer_clean = buffer_wgs[["buffer"]].copy()
buffer_clean = buffer_clean.set_geometry("buffer")

# ===== แยก point stations =====
points = stations_wgs[
    stations_wgs.geometry.type == "Point"
].copy()

# สร้างแผนที่
m = folium.Map(location=[13.75, 100.5], zoom_start=10)

# ===== plot buffer =====
folium.GeoJson(
    buffer_clean.to_json(),
    name="Service Area",
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "blue",
        "weight": 2,
        "fillOpacity": 0.25
    }
).add_to(m)

# ===== plot stations =====
for _, row in points.iterrows():

    popup = row.get("name", "Unknown")

    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=6,
        color="red",
        fill=True,
        fill_color="yellow",
        fill_opacity=0.9,
        popup=popup
    ).add_to(m)

# layer control
folium.LayerControl().add_to(m)

m

## **สรุปผลการเรียนรู้**
จากการทำปฏิบัติการ Lab 1 เรื่องการวิเคราะห์พื้นที่ให้บริการของสถานีขนส่งสาธารณะด้วย GeoPandas ได้เรียนรู้ประเด็นสำคัญดังนี้:

1.  **การจัดการข้อมูลเชิงพื้นที่:** ได้เรียนรู้การดึงข้อมูลจริงจาก OpenStreetMap ผ่าน `osmnx` และแปลงข้อมูลให้อยู่ในรูปแบบ GeoDataFrame ซึ่งทำให้เข้าใจโครงสร้างข้อมูลแบบ Vector (Point/Polygon)
2.  **ความสำคัญของ CRS:** ได้เข้าใจอย่างลึกซึ้งว่าก่อนการคำนวณระยะทางหรือพื้นที่ (เช่นการทำ Buffer) จำเป็นต้องแปลงระบบพิกัดภูมิศาสตร์ (Geographic CRS) ให้เป็นระบบพิกัดโปรเจกชัน (Projected CRS) เช่น EPSG:32647 เพื่อความถูกต้องของหน่วยเมตริก
3.  **การวิเคราะห์ด้วย Buffer และ Spatial Join:**
    * **Buffer:** ช่วยให้เห็นภาพขอบเขตการให้บริการ ซึ่งสามารถนำไปประยุกต์ใช้ในการหาพื้นที่ที่เข้าถึงระบบขนส่งได้
    * **Spatial Join:** เป็นเครื่องมือในการเชื่อมโยงข้อมูลที่มีความสัมพันธ์กันทางภูมิศาสตร์ (เช่น สถานีอยู่ในเขตไหน) ซึ่งต่างจากการ Join ด้วย Key ID ปกติ
4.  **การนำเสนอข้อมูล:** การใช้ Folium ช่วยให้การนำเสนอข้อมูลมีความน่าสนใจและใช้งานได้จริงมากขึ้น เนื่องจากผู้ใช้สามารถโต้ตอบกับแผนที่ได้


### ตอบคำถามท้าย Lab

1. อธิบายความแตกต่างระหว่าง Spatial Join และ Attribute Join
* **Attribute Join:** เป็นการเชื่อมโยงตารางข้อมูลสองตารางโดยใช้ **คอลัมน์ที่มีค่าเหมือนกัน (Common Key/ID)** เป็นตัวเชื่อม เช่น เชื่อมตารางรหัสไปรษณีย์เข้ากับตารางชื่อจังหวัด
* **Spatial Join:** เป็นการเชื่อมโยงข้อมูลโดยใช้ **ความสัมพันธ์ทางตำแหน่งภูมิศาสตร์ (Location)** เป็นเงื่อนไข เช่น จุดนี้อยู่ภายในโพลิกอนไหน (within) หรือ เส้นนี้ตัดผ่านพื้นที่ไหน (intersects) โดยไม่จำเป็นต้องมี Common Key

2. ทำไมเราต้องแปลงระบบพิกัดก่อนใช้ buffer() ใน GeoPandas?
* เพราะระบบพิกัดเริ่มต้นมักเป็น WGS84 (EPSG:4326) ซึ่งมีหน่วยเป็น **องศา (Degrees)** การกำหนด buffer(5) จะหมายถึง 5 องศา (ประมาณ 550 กิโลเมตร) ซึ่งผิดเพี้ยนไปจากความต้องการ
* การแปลงเป็น Projected CRS (เช่น EPSG:32647 สำหรับไทย) จะเปลี่ยนหน่วยเป็น **เมตร** ทำให้การใส่ค่า buffer(5000) ได้ผลลัพธ์ที่ถูกต้องคือ 5 กิโลเมตร

3. หากต้องการวิเคราะห์พื้นที่ให้บริการของสถานีขนส่งในระยะ 10 กิโลเมตร แทน 5 กิโลเมตร ต้องแก้ไขส่วนใดของโค้ด?
* ต้องแก้ไขตัวเลขในฟังก์ชัน `.buffer()`
* จากเดิม `stations_proj.buffer(5000)` ให้เปลี่ยนเป็น `stations_proj.buffer(10000)` (หน่วยเป็นเมตร)

4. วิธีใดที่เหมาะสมที่สุดในการแสดงผลข้อมูลเชิงพื้นที่ในรูปแบบ Interactive Map? เพราะเหตุใด?
* การใช้ไลบรารี **Folium** (หรือไลบรารีอื่นๆ เช่น Plotly Mapbox, Kepler.gl) เหมาะสมที่สุด
* **เหตุผล:** ผู้ใช้งานสามารถซูมเข้า-ออก (Zoom) เลื่อนแผนที่ (Pan) และคลิกที่จุดข้อมูลเพื่อดูรายละเอียด (Popup) ได้ ซึ่งช่วยให้เข้าใจบริบทของพื้นที่ได้ดีกว่าแผนที่ภาพนิ่ง (Static Map) โดยเฉพาะเมื่อมีข้อมูลจำนวนมาก

5. หากพบว่า Buffer ที่สร้างมีขนาดไม่ถูกต้อง อาจเกิดจากสาเหตุใดได้บ้าง? และสามารถแก้ไขได้อย่างไร?
* **สาเหตุ:** ส่วนใหญ่เกิดจากการใช้ **ระบบพิกัด (CRS) ผิดประเภท** คือใช้ระบบพิกัดภูมิศาสตร์ (Lat/Lon) ซึ่งมีหน่วยเป็นองศาในการคำนวณ Buffer
* **การแก้ไข:** ให้ทำการแปลง CRS (`.to_crs()`) ไปเป็นระบบพิกัดแบบ Projected (เช่น UTM) ที่มีหน่วยเป็นเมตรหรือฟุต ก่อนที่จะเรียกใช้ฟังก์ชัน `buffer()`

